# Проект по дисциплине "Прикладные задачи анализа данных"
## Выполнили:
### Гуревич Кирилл, Гречушкин Михаил, Меркудинов Сергей
### Группа ИАД-4

### Постановка задачи

В Интернете существует большое количество специальных сервисов вопросов и ответов (StackOverflow, Тостер, Ответы@Mail.Ru, Quora и другие). Пользователи этих порталов зачастую задают идентичные вопросы или вопросы по достаточно близким темам. Модели, определяющие идентичные вопросы, позволяют пользователям быстрее найти ответы на интересующие их вопросы и уменьшить количество однотипных вопросов.

Целью данного проекта является создание модели, способной определить, являются ли два вопроса повторяющимися (однотипными).

Kaggle: https://www.kaggle.com/c/quora-question-pairs

### Исходные данные

Загрузим оба датасета и посмотрим основные характеристики:
* количество объектов
* количество признаков
* описание признаков


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import log_loss
from sklearn.cross_validation import train_test_split

pal = sns.color_palette()

%matplotlib inline

In [ ]:
train_df = pd.read_csv("../input/train.csv", index_col = 0, error_bad_lines = False, sep = ',')
test_df = pd.read_csv("../input/test.csv", index_col = 0, error_bad_lines = False, sep = ',')

#kostyly
for i in range(len(train_df['question1'])-1, 0, -1):
    if len(train_df['question1'][i]) < 5:
        train_df = train_df.drop(train_df.index[i])

train_df = train_df.reset_index()
for i in range(len(train_df['question2'])-1, 0, -1):
    if len(str(train_df['question2'][i])) < 5:
        train_df = train_df.drop(train_df.index[i])

In [ ]:
print("Тренировочный датасет")
print("Объеков: " + str(train_df.shape[0])+ "\n" 
      "Признаков: " + str(train_df.shape[1]))
print('-'*20)

print("Тестовый датасет")
print("Объектов: " + str(test_df.shape[0])+ "\n" 
      "Признаков: " + str(test_df.shape[1]))

Из результатов выше заметно, что тестовый датасет намного превосходит тренировочный (2,3 миллиона объектов против 404 тысяч), а также то, что в тестовом всего 2 признака. <br> Посмотрим на содержание датасетов

In [ ]:
train_df.head()[0:3]

In [ ]:
test_df.head()[0:3]

Видно, что в тестовом наборе данных отсутствуют признаки qid1, qid2(индексы вопросов) и is_duplicate. Также заметно, что некоторая часть вопросов в тестовом наборе данных сгенерирована автоматически. К примеру, вопросы "What foods fibre?" и "How their can I start reading?" явно лишены какого-либо смысла. 

#### Описание признаков
Описание всех присутствующих признаков:
* <b>id - </b>идентификатор пары вопросов 
* <b>qid1, qid2 - </b>уникальные идентификаторы для каждого вопроса (присутствуют только в данных для обучения)
* <b>question1, question2 - </b>тексты вопросов
* <b>is_duplicate - </b>целевая переменная (отсутствует в тестовом наборе). Принимает значение 1, если вопросы повторяют друг друга, 0 - в противном случае

### Визуализация данных

Для начала посмотрим на общее количество вопросов и соотношение пар повторяющихся и неповторяющихся вопросов.

In [10]:
qids = pd.Series(train_df.qid1.tolist() + train_df.qid2.tolist())
print("Всего уникальных вопросов : " + str(len(np.unique(qids))))
print("Доля повторяющихся пар вопросов: " + str(round(train_df.is_duplicate.mean()*100, 1)) + "%")

In [ ]:
target_val_count = train_df.is_duplicate.value_counts()
plt.figure(figsize=(8, 5))
sns.barplot(target_val_count.index, target_val_count.values, alpha = 0.8)
plt.xlabel("Повторяющиеся пары вопросов", fontsize = 14)
plt.ylabel("Количество пар", fontsize = 14)

Посмотрим на то, сколько раз чаще всего встречаются вопросы.

In [ ]:
plt.figure(figsize=(11, 6))
plt.yscale('log')
plt.hist(qids.value_counts(), bins = 25, alpha = 0.8)
plt.xlabel('Количество раз', fontsize = 14)
plt.ylabel('Количество вопросов', fontsize = 14)

Из гистограммы видно, что большая часть вопросов встречается в тренировочном датасете от 1 до 60 раз. Примечательно, что есть вопросы, встречающиеся более 100 раз.

Посмотрим теперь на распределения количества слов и количества символов в вопросах.

In [ ]:
train_questions = pd.Series(train_df['question1'].tolist() + train_df['question2'].tolist()).astype(str)
test_questions = pd.Series(test_df['question1'].tolist() + test_df['question2'].tolist()).astype(str)
train_dist = train_questions.apply(len)
test_dist = test_questions.apply(len)

plt.figure(figsize = (16, 5))
plt.hist(train_dist, bins = 200, color = pal[1], range = [0, 200], normed = True, label = 'Train', alpha = 0.8)
plt.hist(test_dist, bins = 200, color = pal[2], range = [0, 200], normed = True, label = 'Test', alpha = 0.5)
plt.xlabel("Количество символов", fontsize = 14)
plt.ylabel("Вероятность", fontsize = 14)
plt.legend()
plt.show()

Видно, что почти все вопросы содержат от 20 до 150 символов. Примечталельно то, что в тренировочном датасете вероятность резко падает при достижении отметки в 150 символов.

In [ ]:
train_dist = train_questions.apply(lambda x: len(x.split(' ')))
test_dist = test_questions.apply(lambda x: len(x.split(' ')))
plt.figure(figsize = (16, 5))
plt.hist(train_dist, bins = 40, range = [0, 40], color = pal[1], normed = True, label = 'Train')
plt.hist(test_dist, bins = 40, range = [0, 40], color = pal[2], normed = True, label = 'Test', alpha = 0.5)
plt.xlabel('Количество слов', fontsize = 14)
plt.ylabel('Вероятность', fontsize = 14)
plt.legend()
plt.show()

Из последней гистограммы видно, что большинство пользователей портала Quora задают не очень длинные вопросы. Чаще всего длина вопрос колеблется от 5 до 20-25 слов.

С помощью wordcloud можно наглядно представить наиболее часто встречающиеся слова и словосочетания.

In [ ]:
from wordcloud import WordCloud
cloud = WordCloud(width=1440, height=1080).generate(" ".join(train_questions.astype(str)))
plt.figure(figsize=(15, 10))
plt.imshow(cloud)
plt.axis('off')

### Предобработка данных

Для предобработки исходных данных необходимо выполнить избавление от стоп-слов, токенизацию и стемминг.

In [ ]:
# токенизация
import nltk

trainDfTokenized = train_df.copy()
trainDfTokenized['question1'] = [nltk.word_tokenize(i) for i in trainDfTokenized['question1'].get_values()]
trainDfTokenized['question2'] = [nltk.word_tokenize(i) for i in trainDfTokenized['question2'].get_values()]

In [ ]:
# стемминг и удаление стоп-слов
from nltk.stem.wordnet import WordNetLemmatizer
import string
from nltk.corpus import stopwords

def stemmingForListOfWords(lst):
    lmtzr = WordNetLemmatizer()
    stops = list(stopwords.words("english"))
    punct = list(string.punctuation)
    
    lst = [i for i in lst if i.lower() not in stops+punct]
    lst = [lmtzr.lemmatize(i) for i in lst]
    return lst

trainDfTokenized['question1'] = [stemmingForListOfWords(i) for i in trainDfTokenized['question1'].get_values()]
trainDfTokenized['question2'] = [stemmingForListOfWords(i) for i in trainDfTokenized['question2'].get_values()]

In [ ]:
trainDfTokenized[0:3]

In [ ]:
def compare2questions(row):
    if len(row['question1']) == 0 or len(row['question2']) == 0:
        return 0
    else:
        sharedWordsFrom1To2 = [w for w in row['question1'] if w in row['question2']]
        sharedWordsFrom2To1 = [w for w in row['question2'] if w in row['question1']]
        
        return (len(sharedWordsFrom1To2) + len(sharedWordsFrom2To1)) / (len(row['question1']) + len(row['question2']))

In [ ]:
firstMetric = trainDfTokenized.apply(compare2questions, axis = 1, raw = True)

In [ ]:
from collections import Counter

# If a word appears only once, we ignore it completely (likely a typo)
# Epsilon defines a smoothing constant, which makes the effect of extremely rare words smaller
def get_weight(count, eps=10000, min_count=2):
    if count < min_count:
        return 0
    else:
        return 1 / (count + eps)


train_qs = pd.Series(train_df['question1'].tolist() + train_df['question2'].tolist()).astype(str)
test_qs = pd.Series(test_df['question1'].tolist() + test_df['question2'].tolist()).astype(str)

eps = 5000 
words = (" ".join(train_qs)).lower().split()
counts = Counter(words)
weights = {word: get_weight(count) for word, count in counts.items()}

In [ ]:
from nltk.corpus import stopwords

stops = set(stopwords.words("english"))

def tfidf_word_match_share(row):
    q1words = {}
    q2words = {}
    for word in str(row['question1']).lower().split():
        if word not in stops:
            q1words[word] = 1
    for word in str(row['question2']).lower().split():
        if word not in stops:
            q2words[word] = 1
    if len(q1words) == 0 or len(q2words) == 0:
        # The computer-generated chaff includes a few questions that are nothing but stopwords
        return 0
    
    shared_weights = [weights.get(w, 0) for w in q1words.keys() if w in q2words] + [weights.get(w, 0) for w in q2words.keys() if w in q1words]
    total_weights = [weights.get(w, 0) for w in q1words] + [weights.get(w, 0) for w in q2words]
    
    R = np.sum(shared_weights) / np.sum(total_weights)
    return R

In [ ]:
secondMetric = train_df.apply(tfidf_word_match_share, axis=1, raw=True)

In [ ]:
print(secondMetric[0:3])

### Обучение моделей

Для решения поставленной задачи попробуем использовать XGBoost и логистическую регрессию. После обучения моделей сравним их качество и сформулируем выводы.

#### XGBoost

In [ ]:
# First we create our training and testing data
x_train = pd.DataFrame()
x_test = pd.DataFrame()
x_train['word_match'] = firstMetric
x_train['tfidf_word_match'] = secondMetric
y_train = trainDfTokenized['is_duplicate'].get_values()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_train, y_train,\
                                                    test_size = 0.25, random_state = 171)

In [ ]:
import xgboost as xgb

# Set our parameters for xgboost
params = {}
params['objective'] = 'binary:logistic'
params['eval_metric'] = 'logloss'
params['eta'] = 0.02
params['max_depth'] = 4

d_train = xgb.DMatrix(X_train, label=y_train)
d_valid = xgb.DMatrix(X_test, label=y_test)

watchlist = [(d_train, 'train'), (d_valid, 'valid')]

bst = xgb.train(params, d_train, 400, watchlist, early_stopping_rounds=50, verbose_eval=10)

#### Логистическая регрессия

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

In [ ]:
Bag = CountVectorizer(max_df=0.999, min_df=50, max_features=300, 
                                      analyzer='char', ngram_range=(1,2), 
                                      binary=True, lowercase=True, stop_words='english')

In [ ]:
Bag.fit(pd.Series(trainDfTokenized['question1'] + trainDfTokenized['question2']).astype(str).unique())

In [ ]:
question1 = Bag.transform(pd.Series(trainDfTokenized['question1']).astype(str))
question2 = Bag.transform(pd.Series(trainDfTokenized['question2']).astype(str))

In [ ]:
question1.shape

In [ ]:
X = -(question1 != question2).astype(int)
y = train_df['is_duplicate'].values
y.shape

In [ ]:
logisticRegressor = LogisticRegression(C=0.1, solver='sag', 
                                                    class_weight={1: 0.472008228977, 0: 1.30905513329})
logisticRegressor.fit(X, y)

In [ ]:
predict = logisticRegressor.predict_proba(X)
predict

In [ ]:
print("LogLoss score: " + str(log_loss(y,predict[:,1])))

### Выводы

В результате выполнения данного проекта было использовано два классификатора для поиска однотипных вопросов: логистическая регрессия и Xgboost. Наилучший результат показала классификация с помощью Xgboost с результатом valid-logloss:<b>0.47798</b>. К сожалению, на текущий момент не удалось обработатать тестирующий датасет и загрузить результаты на Kaggle ввиду недостаточных объемов оперативной памяти как на ноутбуках, так и доступного объема в облаке Kaggle. Однако оценка по доступным данным позволяет судить, что цель работы успешно достигнута.